# Regression map of radiative heating induces by moisture and temperature

## Import package

In [1]:
import h5py
import numpy as np
import matplotlib.pyplot as plt

## Load data

In [2]:
# Load composited moisture and temperature data
t_comp = {}
q_comp = {}

with h5py.File("/work/b11209013/2025_Research/regression/IMERG_ERA5.h5", "r") as f:
    t_comp_grp = f["t_composite"]
    q_comp_grp = f["q_composite"]

    for key in t_comp_grp.keys():
        t_comp[key] = np.array(t_comp_grp.get(key))
        q_comp[key] = np.array(q_comp_grp.get(key))

# load linear response functions
with h5py.File("/work/b11209013/2025_Research/MSI/Rad_Stuff/LRF.h5", "r") as f:
    q_lw = np.array(f.get("q_lw"))[::-1, ::-1]
    q_sw = np.array(f.get("q_sw"))[::-1, ::-1]
    t_lw = np.array(f.get("t_lw"))[::-1, ::-1]
    t_sw = np.array(f.get("t_sw"))[::-1, ::-1]


## Compute Corresponding radiative heating

In [3]:
q_lw_prof = {}
q_sw_prof = {}
t_lw_prof = {}
t_sw_prof = {}

for key in t_comp.keys():
    q_lw_prof[key] = q_lw @ q_comp[key]
    q_sw_prof[key] = q_sw @ q_comp[key]
    t_lw_prof[key] = t_lw @ t_comp[key]
    t_sw_prof[key] = t_sw @ t_comp[key]

## Save radiative heating profile

In [4]:
with h5py.File("/work/b11209013/2025_Research/regression/IMERG_MOISTRAD.h5", "w") as f:

    q_lw_grp = f.create_group("q_lw_composite")
    q_sw_grp = f.create_group("q_sw_composite")
    t_lw_grp = f.create_group("t_lw_composite")
    t_sw_grp = f.create_group("t_sw_composite")

    for key in q_lw_prof.keys():
        q_lw_grp.create_dataset(key, data=q_lw_prof[key])
        q_sw_grp.create_dataset(key, data=q_sw_prof[key])
        t_lw_grp.create_dataset(key, data=t_lw_prof[key])
        t_sw_grp.create_dataset(key, data=t_sw_prof[key])

